# Google Earth Engine Panel Data Creation

## Initialize

In [2]:
# !pip install geemap
#!pip install ee

In [3]:
# !pip install uszipcode

In [4]:
#GEE specific
import ee
import geemap
import math

#plotting and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from time import time


In [5]:
#Initialize Google Earth Engine
#ee.Authenticate() #just needed the 1st time
ee.Initialize()

In [6]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()

## Load Feature Collection - Shapefiles

In [7]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))
sc_county = counties.filter(ee.Filter.eq('NAME', 'Santa Clara'))

In [8]:
la_county, sc_county

(<ee.featurecollection.FeatureCollection at 0x7f9359140250>,
 <ee.featurecollection.FeatureCollection at 0x7f9359140430>)

In [9]:
#Income Data
la_county_income = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome-final")

## Load NAIP Imagery

In [10]:
def apply_3bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image

def add_neighborhood_bands(image):
    bands = ['R', 'G', 'B', 'N']
    for band in bands:
        image = apply_3bands(image, band)
    return image
    
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image
     

In [11]:
def get_images(param_dict):
    source_image_collection = params['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-01-01', f'{year}-12-31')\
                                .select(['R','G','B','N'])\
                                .median().clip(counties[county])
        images.append(image)
        images_with_3band = list(map(add_neighborhood_bands, images))
        images_with_NDVI = list(map(add_NDVI, images_with_3band))
    return dict(zip(image_names, images_with_NDVI))

    
    

In [12]:
params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2010,2012,2014,2016,2018,2020],
        'counties': {'la_county': la_county, 'sc_county': sc_county}
         }

images = get_images(params)
images

{'2010_la_county': <ee.image.Image at 0x7f936896df40>,
 '2010_sc_county': <ee.image.Image at 0x7f9368971160>,
 '2012_la_county': <ee.image.Image at 0x7f9368971340>,
 '2012_sc_county': <ee.image.Image at 0x7f9368971520>,
 '2014_la_county': <ee.image.Image at 0x7f9368971700>,
 '2014_sc_county': <ee.image.Image at 0x7f93689718e0>,
 '2016_la_county': <ee.image.Image at 0x7f9368971ac0>,
 '2016_sc_county': <ee.image.Image at 0x7f9368971ca0>,
 '2018_la_county': <ee.image.Image at 0x7f9368971e80>,
 '2018_sc_county': <ee.image.Image at 0x7f93689740a0>,
 '2020_la_county': <ee.image.Image at 0x7f9368974280>,
 '2020_sc_county': <ee.image.Image at 0x7f9368974460>}

In [13]:
#see what bands each image has:
images['2020_la_county'].bandNames().getInfo()

['R',
 'G',
 'B',
 'N',
 'R_Entropy',
 'R_Contrast',
 'R_Gearys',
 'G_Entropy',
 'G_Contrast',
 'G_Gearys',
 'B_Entropy',
 'B_Contrast',
 'B_Gearys',
 'N_Entropy',
 'N_Contrast',
 'N_Gearys',
 'NDVI']

In [14]:
#test to see if NAIP plot looks correct
Map.addLayer(images['2020_la_county'], {}, 'NAIP Test')

In [15]:
#test to see if NDVI plot looks correct (define a different location in santa clara county just for visibility)
source_image = images['2020_sc_county']

test_ndvi = source_image.normalizedDifference(['N', 'R']).rename('2020_NDVI')
test_ndvi

In [16]:
Map.addLayer(test_ndvi,{min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NDVI Test')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load Labeled Data

In [17]:
## Loading feature collections from Google Earth Engine

#water = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance")
water_training = ee.FeatureCollection("projects/california-lawn-detection/assets/water_training")
trees_training = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_training")
grass_training = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_training")
turf_training = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_training")
#pv_training = ee.FeatureCollection("projects/california-lawn-detection/assets/pv_training")
impervious_training = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_training").limit(50)
soil_training = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_training").limit(50)

LABELED_SET = water_training.merge(trees_training).merge(grass_training).merge(turf_training).merge(impervious_training).merge(soil_training)

In [18]:
water_test = ee.FeatureCollection("projects/california-lawn-detection/assets/water_test")
vegetation_trees_test = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_test")
vegetation_grass_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_test")
turf_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_test")
#pv_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/pv_test")
impervious_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_test")
soil_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_test")

TEST_SET = water_test.merge(vegetation_trees_test).merge(vegetation_grass_test).merge(turf_test).merge(impervious_test).merge(soil_test)

## Build Training Set

In [19]:
training_image_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2020],
        'counties': {'lacounty': la_county}
         }

TRAINING_IMAGE = get_images(training_image_params)['2020_lacounty']

In [20]:
# Overlay the points on the imagery to get training.
LABEL = 'landcover'
BANDS = ['R', 
         'G', 
         'B', 
         'N', 
         'NDVI',
         'R_Entropy',
         'R_Contrast',
         'R_Gearys',
         'G_Entropy',
         'G_Contrast',
         'G_Gearys',
         'B_Entropy',
         'B_Contrast',
         'B_Gearys',
         'N_Entropy', 
         'N_Contrast', 
         'N_Gearys']

train_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
  'collection': LABELED_SET,
  'properties': [LABEL],
  'scale': 1
})

test_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
  'collection': TEST_SET,
  'properties': [LABEL],
  'scale': 1
})

In [21]:
set(BANDS)==set(TRAINING_IMAGE.bandNames().getInfo())

True

In [22]:
print("Training Set Size in Pixels", train_data.aggregate_count('R').getInfo())

Training Set Size in Pixels 169069


In [23]:
print("Test Set Size in Pixels", test_data.aggregate_count('R').getInfo())

Test Set Size in Pixels 97964


## Machine Learning Model

In [24]:
clf = ee.Classifier.smileRandomForest(numberOfTrees = 200, minLeafPopulation = 5, bagFraction= 0.7)\
                   .train(train_data, LABEL, BANDS)
clf

In [25]:
training_image_classified = TRAINING_IMAGE.select(BANDS)\
                                          .classify(clf)


In [26]:
legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

Map.addLayer(training_image_classified, {'min': 1, 'max': 7, 'palette': legend_colors}, 'Classification')

In [27]:
training_image_classified.bandNames().getInfo()

['classification']

In [28]:
Map

Map(bottom=3598.0, center=[33.0532776963449, -104.56386550767168], controls=(WidgetControl(options=['position'…

In [30]:
la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [31]:
polygon = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', '90027'))

In [40]:
zip_classified = training_image_classified.clip(polygon)

In [41]:
Map.addLayer(zip_classified,{'min': 1, 'max': 7, 'palette': legend_colors}, 'Zip Classification')

In [42]:
Map

Map(bottom=52630.0, center=[34.066311964721045, -118.27194213867189], controls=(WidgetControl(options=['positi…

In [47]:
combined_img = ee.ImageCollection([images['2020_la_county'], zip_classifed]).mosaic()

In [74]:
# aoi = ee.Geometry.Polygon(
#   [[[-118.31338621939321, 34.15192499486403],
#        [-118.31339067849423, 34.15192499486403],
#        [-118.31339513759531, 34.15192499486403],
#        [-118.31343081040374, 34.15192053576297]]], None, False)


aoi = ee.Geometry.Polygon(
  [[[-118.4, 34.2],
       [-118.4, 34.1],
       [-118.3, 34.1],
       [-118.3, 34.2]]], None, False)

In [75]:
img = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2020-01-01', '2020-12-31')
    .select(['R', 'G', 'B'])
).median()

band_arrs = img.sampleRectangle(region=aoi)

red_band = band_arrs.get('R')
red_band.getInfo()


# # Get individual band arrays.
# band_arr_b4 = band_arrs.get('R')
# band_arr_b5 = band_arrs.get('G')
# band_arr_b6 = band_arrs.get('B')

# # Transfer the arrays from server to client and cast as np array.
# np_arr_b4 = np.array(band_arr_b4.getInfo())
# np_arr_b5 = np.array(band_arr_b5.getInfo())
# np_arr_b6 = np.array(band_arr_b6.getInfo())
# print(np_arr_b4.shape)
# print(np_arr_b5.shape)
# print(np_arr_b6.shape)

# # Expand the dimensions of the images so they can be concatenated into 3-D.
# np_arr_b4 = np.expand_dims(np_arr_b4, 2)
# np_arr_b5 = np.expand_dims(np_arr_b5, 2)
# np_arr_b6 = np.expand_dims(np_arr_b6, 2)
# print(np_arr_b4.shape)
# print(np_arr_b5.shape)
# print(np_arr_b6.shape)

# # Stack the individual bands to make a 3-D array.
# rgb_img = np.concatenate((np_arr_b6, np_arr_b5, np_arr_b4), 2)
# print(rgb_img.shape)

# # Scale the data to [0, 255] to show as an RGB image.
# rgb_img_test = (255*((rgb_img - 100)/3500)).astype('uint8')
# plt.imshow(rgb_img_test)
# plt.show()

[[105]]

In [53]:
combined_img.bandNames().getInfo()

['R',
 'G',
 'B',
 'N',
 'R_Entropy',
 'R_Contrast',
 'R_Gearys',
 'G_Entropy',
 'G_Contrast',
 'G_Gearys',
 'B_Entropy',
 'B_Contrast',
 'B_Gearys',
 'N_Entropy',
 'N_Contrast',
 'N_Gearys',
 'NDVI']

In [51]:
Map.addLayer(combined_img, {}, 'test')

In [52]:
Map

Map(bottom=13362.20703125, center=[34.252676117101515, -117.43835449218751], controls=(WidgetControl(options=[…

In [44]:
polygon.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'version': 1655181821530462,
 'id': 'projects/california-lawn-detection/assets/income_zipcode2019',
 'properties': {'system:asset_size': 2545157},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'GeometryCollection',
    'geometries': [{'type': 'LineString',
      'coordinates': [[-118.31338621939321, 34.15192499486403],
       [-118.31339067849423, 34.15192499486403],
       [-118.31339513759531, 34.15192499486403],
       [-118.31343081040374, 34.15192053576297]]},
     {'type': 'Polygon',
      'coordinates': [[[-118.27521634175552, 34.09686845599582],
        [-118.27513159007081, 34.09697102514422],
        [-118.27514497191856, 34.096979949409146],
        [-118.27511820822201, 34.09701116365621],
        [-118.2750646808163, 34.09707804527545],
        [-118.27506022019841, 34.09708251847623],
        [-118.2750066927745, 34.09714494691368],
        [-118.27497109702833, 34.09718953838423],
        [-118.2749354120623, 34.

In [ ]:
band_arrs = img.sampleRectangle(region=aoi)